In [1]:
import numpy as np
import pprint
import sys
if "../" not in sys.path:
  sys.path.append("../") 
from lib.envs.gridworld import GridworldEnv

In [2]:
pp = pprint.PrettyPrinter(indent=2)
env = GridworldEnv()

In [3]:
def policy_eval(policy, env, discount_factor=1.0, theta=0.00001):
    """
    Evaluate a policy given an environment and a full description of the environment's dynamics.
    
    Args:
        policy: [S, A] shaped matrix representing the policy.
        env: OpenAI env. env.P represents the transition probabilities of the environment.
            env.P[s][a] is a (prob, next_state, reward, done) tuple.
        theta: We stop evaluation one our value function change is less than theta for all states.
        discount_factor: lambda discount factor.
    
    Returns:
        Vector of length env.nS representing the value function.
                
    """
    
    V = np.zeros(env.nS)
    while True:
        V_next = np.zeros(env.nS)
        # check each state
        for s in env.P:
            v = 0
            # check each action
            for a, action_prob in enumerate(policy[s]):
                # get values for each possible next state given action a
                for next_state_prob, next_state, reward, done in env.P[s][a]:
                    V_next[s] += action_prob * next_state_prob * (reward + discount_factor * V[next_state])
        if np.all((V - V_next) < theta):
            break
        V = V_next
    return V

In [36]:
def policy_improvement(env, policy_eval_fn=policy_eval, discount_factor=1.0):
    """
    Policy Improvement Algorithm. Iteratively evaluates and improves a policy
    until an optimal policy is found.
    
    Args:
        env: The OpenAI envrionment.
        policy_eval_fn: Policy Evaluation function that takes 3 arguments:
            policy, env, discount_factor.
        discount_factor: Lambda discount factor.
        
    Returns:
        A tuple (policy, V). 
        policy is the optimal policy, a matrix of shape [S, A] where each state s
        contains a valid probability distribution over actions.
        V is the value function for the optimal policy.
        
    """
    # Start with a random policy
    policy = np.ones([env.nS, env.nA]) / env.nA
    
    while True:
        policy_stable = True
        V = np.array(policy_eval(policy, env, discount_factor))
        for state in env.P:
            chosen_action = np.argmax(policy[state])
            # get next states and moves to get there
            next_states_moves = {env.P[state][a][0][1]:a for a in env.P[state]}
            
            # get the values of the next moves
            next_values = V[next_states_moves.keys()]
            
            # get the best state to go to
            best_state = next_states_moves.keys()[np.argmax(next_values)]
            
            # get the move to go to the best state
            best_action = next_states_moves[best_state]
            if chosen_action != best_action:
                policy_stable = False
            # make the policy for the current state all zeros, then set the best move to probability 1
            policy[state] = np.zeros(np.array(policy[state]).shape)
            policy[state, best_action] = 1
            
        if policy_stable:    
            return policy, V
        #break
    #V = policy_eval(policy, env, discount_factor)
    #return policy, V

In [8]:
print env.P[0]
for a in env.P[0]:
    print env.P[0][a][0][1]

{0: [(1.0, 0, 0.0, True)], 1: [(1.0, 0, 0.0, True)], 2: [(1.0, 0, 0.0, True)], 3: [(1.0, 0, 0.0, True)]}
0
0
0
0


In [37]:
policy, v = policy_improvement(env)
print("Policy Probability Distribution:")
print(policy)
print("")

print("Reshaped Grid Policy (0=up, 1=right, 2=down, 3=left):")
print(np.reshape(np.argmax(policy, axis=1), env.shape))
print("")

print("Value Function:")
print(v)
print("")

print("Reshaped Grid Value Function:")
print(v.reshape(env.shape))
print("")

Policy Probability Distribution:
[[ 0.  0.  0.  1.]
 [ 0.  0.  0.  1.]
 [ 0.  0.  0.  1.]
 [ 0.  0.  0.  1.]
 [ 1.  0.  0.  0.]
 [ 1.  0.  0.  0.]
 [ 1.  0.  0.  0.]
 [ 0.  0.  1.  0.]
 [ 1.  0.  0.  0.]
 [ 0.  0.  0.  1.]
 [ 0.  1.  0.  0.]
 [ 0.  0.  1.  0.]
 [ 1.  0.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  0.  0.  1.]]

Reshaped Grid Policy (0=up, 1=right, 2=down, 3=left):
[[3 3 3 3]
 [0 0 0 2]
 [0 3 1 2]
 [0 1 1 3]]

Value Function:
[ 0. -1. -2. -3. -1. -2. -3. -2. -2. -3. -2. -1. -3. -2. -1.  0.]

Reshaped Grid Value Function:
[[ 0. -1. -2. -3.]
 [-1. -2. -3. -2.]
 [-2. -3. -2. -1.]
 [-3. -2. -1.  0.]]



In [38]:
expected_v = np.array([ 0, -1, -2, -3, -1, -2, -3, -2, -2, -3, -2, -1, -3, -2, -1,  0])
np.testing.assert_array_almost_equal(v, expected_v, decimal=2)